In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

llm.invoke(prompt)

# GPT는 계산을 못 한다.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

def plus(a, b):
    return a + b

# agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION 우리의 tool이 한 개보다,더 많은 input을 가질 수 있다. 
# verbose=True agent가 하는 작업의 모든 과정 및 추론을 보여준다.
agent = initialize_agent(llm=llm, verbose=True , agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION ,tools=[
    StructuredTool.from_function(func=plus, name="Sum Calculator", description="Use this to perform sums of two numbers"),
])

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [ ]:
# Zero-shot React Agent

from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

def plus(inputs):
    a, b = inputs.split(",")
    return float(a) + float(b)

# handle_parsing_errors=True
agent = initialize_agent(llm=llm, verbose=True , agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION , handle_parsing_errors=True, tools=[
    Tool.from_function(func=plus, name="Sum Calculator", description="Use this to perform sums of two numbers. Use this tool by sending a pair of numberts separated by a comma\
        \nExample:1,2"),
])

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [ ]:
# OpenAI Functions Agent
# Pydantic -> python의 데이터 유효성 library, 데이터가 어떤 형태여야 하는지 알려준다.

from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from typing import Type

llm = ChatOpenAI(temperature=0.1)

def plus(a, b):
    return a + b

class CalculatorToolArgsSchema(BaseModel):
    a: int = Field(description="The first number")
    b: int = Field(description="The second number")
    

class CalculatorTool(BaseTool):
    name = "SumCalculator"
    description = """
    Use this to perform sums of two numbers.
    The first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema
    
    def _run(self, a, b):
        return plus(a, b)
    
agent = initialize_agent(llm=llm, verbose=True , agent=AgentType.OPENAI_FUNCTIONS, handle_parsing_errors=True, tools=[
    CalculatorTool(),
])

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)

In [ ]:
# Search Tool

# OpenAI Functions Agent
# Pydantic -> python의 데이터 유효성 library, 데이터가 어떤 형태여야 하는지 알려준다.

from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

from typing import Type

llm = ChatOpenAI(temperature=0.1)

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for.Example query: Stock Market Symbol for Apple Company")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for apple Company
    """
    
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema
    
    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

agent = initialize_agent(llm=llm, verbose=True , agent=AgentType.OPENAI_FUNCTIONS, handle_parsing_errors=True, tools=[
    StockMarketSymbolSearchTool(),
])

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)

In [ ]:
# Search Tool

# OpenAI Functions Agent
# Pydantic -> python의 데이터 유효성 library, 데이터가 어떤 형태여야 하는지 알려준다.
import os
import requests
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool, BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

from typing import Type

llm = ChatOpenAI(temperature=0.1, model_name = "gpt-3.5-turbo-1106")
alpha_vantage_api_key = os.environ.get("GI7UXI89QUWWJFK5")

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for.Example query: Stock Market Symbol for Apple Company")

class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    Example query: Stock Market Symbol for apple Company
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema
    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)

#

class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(description="Stock symbol of the company.Example: AAPL,TSLA")

class CompanyOverviewTool(BaseTool):
    name="CompanyOverview"
    description="""
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    def _run(self, symbol):
        print(requests)
        r = requests.get(f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()

class CompanyIncomeStatementTool(BaseTool):
    name="CompanyIncomeStatement"
    description="""
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()["annualReports"]
    
class CompanyStockPerformanceTool(BaseTool):
    name="CompanyStockperformance"
    description="""
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema
    def _run(self, symbol):
        r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}")
        return r.json()


agent = initialize_agent(llm=llm, verbose=True , agent=AgentType.OPENAI_FUNCTIONS, handle_parsing_errors=True, tools=[
    StockMarketSymbolSearchTool(),
    # 주식 심볼을 아는 데 사용
    CompanyOverviewTool(),
    # 정보를 얻는데 사용
    CompanyIncomeStatementTool(),
    # llm이 사용 안 할 수도 있다. llm이 더 많은 정보가 필요하다고 생각할 때만 사용
    CompanyStockPerformanceTool(),
])

prompt = "Give me financial information on Cloudflare's stock, considering its financials, income statements and stock performance help me analyze if it's a potential good investment."

agent.invoke(prompt)

In [14]:
from langchain.agents import create_sql_agent, agent_types
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(temperature=0.1)

db = SQLDatabase.from_uri("sqlite:///movies.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(llm=llm, toolkit=toolkit, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.invoke("Give me the 5 directors that have the highest grossing films.")

agent.invoke("Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue.")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: directors, movies
Thought:I should query the schema of the directors and movies tables to see what columns I can use to find the highest grossing films.
Action: sql_db_schema
Action Input: directors, movies
Observation: 
CREATE TABLE directors (
	name TEXT, 
	id INTEGER, 
	gender INTEGER, 
	uid INTEGER, 
	department TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from directors table:
name	id	gender	uid	department
James Cameron	4762	2	2710	Directing
Gore Verbinski	4763	2	1704	Directing
Sam Mendes	4764	2	39	Directing
*/


CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	director_id INTEGER DEFAULT 0 NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from movies table:
id	original_title	budget	popularity	release_date	

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue.',
 'output': "The movies with the highest votes but the lowest budgets and their directors' names along with their gross revenue are:\n1. Inception directed by Christopher Nolan with a budget of 160,000,000 and revenue of 825,532,764\n2. The Dark Knight directed by Christopher Nolan with a budget of 185,000,000 and revenue of 1,004,558,444\n3. Avatar directed by James Cameron with a budget of 237,000,000 and revenue of 2,787,965,087\n4. The Avengers directed by Joss Whedon with a budget of 220,000,000 and revenue of 1,519,557,910\n5. Deadpool directed by Tim Miller with a budget of 58,000,000 and revenue of 783,112,979\n6. Interstellar directed by Christopher Nolan with a budget of 165,000,000 and revenue of 675,120,017\n7. Django Unchained directed by Quentin Tarantino with a budget of 100,000,000 and revenue of 425,368,238\n8. Guar

In [ ]:
import sqlite3

conn = sqlite3.connect("movies.sqlite", isolation_level=None)
c = conn.cursor()
c.execute("DROP TABLE IF EXISTS directors;")
c.execute("""
CREATE TABLE directors (
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
name TEXT,
gender INTEGER,
uid INTEGER
);
""")
c.execute("""INSERT INTO directors VALUES
(1, 'James Cameron', 2, 1),
(2, 'Gore Verbinski', 2, 2),
(3, 'Sam Mendes', 2, 3),
(4, 'Christopher Nolan', 2, 4),
(5, 'Andrew Stanton', 2, 5);
""")

c.execute("DROP TABLE IF EXISTS movies;")
c.execute("""
CREATE TABLE movies (
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
original_title TEXT,
budget TEXT,
popularity [REAL],
director_id [INTEGER],
gross_revenue [REAL]
)
""")
c.execute("""INSERT INTO movies VALUES
(1, 'Avatar', 237000000, 150.437577, 1, 2787965087),
(2, 'Pirates of the Caribbean: At World''s End', 300000000, 139.082615, 2, 961000000),
(3, 'Spectre', 245000000, 107.376788, 3, 880674609),
(4, 'The Dark Knight Rises', 250000000, 112.31295, 4, 1084939099),
(5, 'John Carter', 260000000, 43.926995, 5, 284139100);
""")
c.close()